Here we are exploring the Countires of the World by Fernando Lasso.  We will focus on what factors contribute to the GDP growth of each country.  We will start with the visualization.


**I. Data Overview**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)
from sklearn import preprocessing


df = pd.read_csv("../input/countries of the world.csv")

print (df.columns)

num_features = []
cat_features = []
# 0. columns types
print (df.dtypes)

num_features = []
cat_features = []

for dtype, feature in zip(df.dtypes[:], df.columns[:]):
    if dtype == object:
        cat_features.append(feature)
    else:
        num_features.append(feature)
print (cat_features)        
df.tail()

**II. Data Preparation**
- Let' create bin for all the countires based on the GDP per capita

In [ ]:
# Country with the highest GDP
print (df.loc[df['GDP ($ per capita)'].idxmax()])
df['Region'] = df['Region'].map(lambda x: x.strip())
df['Country'] = df['Country'].map(lambda x: x.strip())

df.loc[df.Country=='Western Sahara', 'GDP ($ per capita)'] = 1000

print ('here ',df.loc[df['Country']=='Western Sahara'])
df['GDP_bucket'] = pd.cut(df['GDP ($ per capita)'], [0,1005,3955,12235, 60000], labels=["low", "middle", "upper", "high"])
print (df.tail())





- Let' fill in the missing cells by region and GDP_bucket median 

In [ ]:
cols =   df.columns.values
cols = cols[2:]
cols = cols[:-1]
        
cat_features = cat_features[2:]
geographic_features = ['Area (sq. mi.)','Coastline (coast/area ratio)','Arable (%)','Crops (%)']
social_features = [ 'Population', 'Pop. Density (per sq. mi.)', 'Net migration','Infant mortality (per 1000 births)','GDP ($ per capita)', 'Literacy (%)'
                  ,'Phones (per 1000)','Other (%)','Birthrate ','Deathrate', 'Service', 'Industry','Agriculture']

#print (cat_features)
#print ('hi')
for col in cols:
    if df[col].isnull().sum() == 0:
        continue
    if col == 'Climate':
        median = df.groupby('Region')['Climate'].apply(lambda x: x.mode().max())
        for region in df['Region'].unique():
           df[col].loc[(df[col].isnull())&(df['Region']==region)] = median[region]

    else: 
        if col in geographic_features:    
           try: 
              df[col] = df[col].str.replace(',', '').astype(float)
           except:
              pass
           median = df.groupby('Region')[col].apply(lambda x: x.median() )
           for region in df['Region'].unique():
              df[col].loc[(df[col].isnull())&(df['Region']==region) ] = median[region]
        #elif col in cat_features:
        elif col in social_features:
            
           try: 
              df[col] = df[col].str.replace(',', '').astype(float)
           except:
              pass
           median = df.groupby('GDP_bucket')[col].apply(lambda x: x.median() )
           #print (median)
           median1 = df.groupby(['Region','GDP_bucket'])[col].apply(lambda x: x.median() )
           #print (median1[]
           
           for region in df['Region'].unique():    
              for bucket in df['GDP_bucket'].unique():
                try: 
                   df[col].loc[(df[col].isnull())&(df['Region']==region) &(df['GDP_bucket']==bucket)  ] = median1[region][bucket]
                #         print (median1[bucket][region])
                except:
                   pass
                df[col].loc[(df[col].isnull())&(df['GDP_bucket']==bucket) ] = median[bucket]
                #df = df.fillna(method='ffill')         

        else:
            pass

df= df.fillna(method='ffill')            
df.head()

In [ ]:
 df.isnull().sum()

> III. **Data Visualization**

In [ ]:
# 1. scater plot
dims = (15, 10)
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=dims)
plt.xticks(rotation=60)
sns.catplot(ax = ax, x='Region', y='GDP ($ per capita)',jitter=False, data=df)

In [ ]:
# 2. Boxplot - distribution plot
dims = (15, 10)
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=dims)
plt.xticks(rotation=60)

sns.boxplot(ax = ax, x='Region', y='GDP ($ per capita)', data=df)


In [ ]:
# 3. Violin plot
# Violin plot also shows the distribution of the data
dims = (15, 10)
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=dims)
plt.xticks(rotation=60)

sns.violinplot(ax = ax, x='Region', y='GDP ($ per capita)', data=df)


In [ ]:
# 4. Bar chart
dims = (15, 10)
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=dims)
plt.xticks(rotation=60)

sns.catplot(ax = ax, x='Region', y='GDP ($ per capita)', kind="bar", data=df);



In [ ]:
# Bar chart with occurance counts
dims = (15, 10)
sns.set(font_scale=2) 
fig, ax = plt.subplots(figsize=dims)
plt.xticks(rotation=60)

df['GDP ($ per capita)'] = df['GDP ($ per capita)'].fillna(method='ffill')
# raw count of occurances on Y axis
sns.distplot(df['GDP ($ per capita)'], ax=ax, rug=True, kde=True, hist=False)
sns.distplot(df['GDP ($ per capita)'], ax=ax, rug=True, kde=False, hist=True, norm_hist=False)

In [ ]:
# Bar chart with normalized distribution
dims = (15, 10)
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=dims)
plt.xticks(rotation=60)
ax = sns.distplot(df['GDP ($ per capita)'], norm_hist=False)


In [ ]:
# 5. Heatmap
sns.set(style="white")
dims = (15, 10)
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=dims)

cmap = sns.diverging_palette(220, 10, as_cmap=True)

corr = df.corr()
print (corr)
#df1 = df.iloc[:,1:].head()


sns.heatmap(corr, cmap = cmap, vmax=.3, center=0, square=True, linewidths=.1, cbar_kws={"shrink": .5})

**III. Data Exploration**

 1. Most correlated features:

In [ ]:
# 1. Most correlated features
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20,12))
plt.subplots_adjust(hspace=0.4)

df = df.fillna(method='ffill')

for col in df.columns.values[2:]: 
    if ((col!='Climate')&(col!='GDP_bucket') ) :
       try: 
          df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
       except:
          df[col] = df[col].astype(float)
    
corr_to_gdp = pd.Series()
for col in df.columns.values[2:]:
    if ((col!='GDP ($ per capita)')&(col!='Climate')&(col!='GDP_bucket')):
        corr_to_gdp[col] = df['GDP ($ per capita)'].corr(df[col])   
#print (corr_to_gdp)
abs_corr_to_gdp = corr_to_gdp.abs().sort_values(ascending=False)
#print (abs_corr_to_gdp)
corr_to_gdp = corr_to_gdp.loc[abs_corr_to_gdp.index]
#print (corr_to_gdp)

for i in range(2):
    for j in range(3):
        sns.regplot(x=corr_to_gdp.index.values[i*3+j], y='GDP ($ per capita)', data=df,ax=axes[i,j], fit_reg=False, marker='.')
        title = 'correlation='+str(corr_to_gdp[i*3+j])
        axes[i,j].set_title(title)
axes[1,2].set_xlim(0,102)
plt.show()



2. Countries with the highest GDP per capita by region:

In [ ]:
fig, ax = plt.subplots(figsize=(28,11))

grouped = df.groupby(['Region'])['Country','GDP ($ per capita)']

group_sorted = (grouped.apply(pd.DataFrame.sort_values, 'GDP ($ per capita)') )

top_three = group_sorted.groupby('Region').tail(3)
top_three = top_three.reset_index()


sns.barplot(x='Country',y='GDP ($ per capita)',hue='Region',data=top_three, palette='Set1')
ax.set_xlabel(ax.get_xlabel(),labelpad=15)
ax.set_ylabel(ax.get_ylabel(),labelpad=30)
ax.xaxis.label.set_fontsize(16)
ax.yaxis.label.set_fontsize(16)
plt.xticks(rotation=90)
plt.show()



**IV. GDP per capita prediction and feature importance**

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X = df[['Population', 'Area (sq. mi.)',
       'Pop. Density (per sq. mi.)', 'Coastline (coast/area ratio)',
       'Net migration', 'Infant mortality (per 1000 births)', 'Literacy (%)', 'Phones (per 1000)', 'Arable (%)',
       'Crops (%)', 'Other (%)', 'Climate', 'Birthrate', 'Deathrate',
       'Agriculture', 'Industry', 'Service']]

for col in X.columns.values: 
    try: 
       X[col] = X[col].astype(str).str.replace(',', '.').astype(float)
    except:
       X[col] = X[col].astype(float)




y = df['GDP ($ per capita)']
y = y.convert_objects(convert_numeric=True)
print (y.dtypes)
print (X[['Pop. Density (per sq. mi.)','Coastline (coast/area ratio)','Net migration']].head())
dims = (15, 10)
fig, ax = plt.subplots(figsize=dims)


# fit model no training data
model = XGBRegressor()
model.fit(X, y)
plot_importance(model, ax = ax)
plt.show()
# feature importance
print(model.feature_importances_)
# Prediction
model1 = XGBRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

model1.fit(X_train, y_train)
# make predictions for test data and evaluate
train_pred_Y = model.predict(X_train)
y_pred = model1.predict(X_test)

predictions = [round(value) for value in y_pred]
print (len(predictions))
print (len(y_test))
from sklearn.metrics import r2_score
print ('r square: ',r2_score(y_test, predictions) )


In [ ]:
from sklearn.feature_selection import SelectFromModel


thresholds = [0.06, 0.07]
for thresh in thresholds:
   selection = SelectFromModel(model1, threshold=thresh, prefit=True)
   select_X_train = selection.transform(X_train)
   selection_model = XGBRegressor()
   selection_model.fit(select_X_train, y_train)
   select_X_test = selection.transform(X_test)
   y_pred = selection_model.predict(select_X_test)
   predictions = [round(value) for value in y_pred]
   print ('r square: ',r2_score(y_test, predictions) )



**V. Result visualization**

In [ ]:
sns.set(style="white")
dims = (25, 20)
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=dims)

print (len(y_test))
print  (len(y_pred))
y_pred = pd.Series(y_pred.clip(0, y_pred.max()), index=y_test.index, name="Predicted GDP per capita")


data = df.loc[y_test.index]
data= pd.concat([data, y_pred], axis=1, sort=False)
label = data['Country']
print (label)

ax = sns.regplot(x="GDP ($ per capita)", y="Predicted GDP per capita", data=data)
plt.legend(loc=4,prop={'size': 24})  
ax.set_xlabel('GDP ($ per capita) ground truth',labelpad=40)
ax.set_ylabel('GDP ($ per capita) predicted',labelpad=40)
ax.xaxis.label.set_fontsize(24)
ax.yaxis.label.set_fontsize(24)
ax.tick_params(labelsize=12)

for i in range(0,y_test.shape[0]):

        plt.text(y_test.iloc[i]+200, y_pred.iloc[i]-200, label.iloc[i], size='small')


